# CO2-eq Analysis

_Dataset_: Supplementary data for Megill et al. (2024): "Alternative climate metrics to the Global Warming Potential are more suitable for assessing aviation non-CO2 effects"

_Authors_:

- Liam Megill (1, 2), https://orcid.org/0000-0002-4199-6962   
- Kathrin Deck (2)  
- Volker Grewe (1, 2), https://orcid.org/0000-0002-8012-6783  

_Affiliation (1)_: Deutsches Zentrum für Luft- und Raumfahrt (DLR), Institut für Physik der Atmosphäre, Oberpfaffenhofen, Germany

_Affiliation (2)_: Delft University of Technology (TU Delft), Faculty of Aerospace Engineering, Section Aircraft Noise and Climate Effects (ANCE), Delft, The Netherlands

_Corresponding author_: Liam Megill, liam.megill@dlr.de

_doi_: https://doi.org/10.1038/s43247-024-01423-6

---


### Summary
This notebook analyses the temporal stability (REQ 2) of climate metrics using CO2-eq trajectories and the resulting CO2 multipliers. Specifically, Figure 2 and Extended Data Figure 2 of the linked paper are created.

### Linked data
- `CO2eq/CORSIA`: CORSIA scenario of Grewe et al. (2021), extended by 0.5% annual growth rate from 2100 until 2200
- `CO2eq/COVID15s`: COVID15s scenario of Grewe et al. (2021), extended by 0.5% annual growth rate from 2100 until 2200
- `CO2eq/CurTec`: CurTec scenario of Grewe et al. (2021), extended by 0.8% annual growth rate from 2100 until 2200
- `CO2eq/Fa1`: Fa1 scenario of IPCC (1999), extended by 0.8% annual growth rate from 2100 until 2200
- `CO2eq/FP2050`: FP2050 scenario of Grewe et al. (2021), extended by 0.5% annual growth rate from 2100 until 2200
- `CO2eq/E_bg_new_scen.txt`: Global aviation emission scenario input file for AirClim

---

### Copyright

Copyright © 2024 Liam Megill

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

# define plotting style
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']
sns.set_context("paper", rc={"lines.linewidth": 2})
sns.set_style("ticks")

## Function definitions

In [ ]:
def load_rf_dt(directory):
    """Load the radiative forcing (RF) and temperature change (dT) responses of a given fleet.

    Args:
        directory (_str_): Path to data directory.

    Returns:
        rf_arr (_np.ndarray_): Array (7, len(years)) of radiative forcing results for each emission species
        dt_arr (_np.ndarray_): Array (7, len(years)) of temperature change results for each emission species
        years (_np.ndarray_): Array of response years from AirClim
    """
    emission_species = ["CO2", "H2O", "O3", "CH4", "Cont", "PMO"]
    years = np.loadtxt(directory + "RF_CO2_taumean_rfmean.txt", skiprows=2, usecols=0)
    rf_arr = np.zeros((7, len(years)))  # define empty RF array
    dt_arr = np.zeros((7, len(years)))  # define empty dT array
    # populate rf_arr and dt_arr
    for i in range(len(emission_species)):
        rf_name = "RF_" + emission_species[i] + "_taumean_rfmean.txt"
        dt_name = "dT_" + emission_species[i] + "_taumean_rfmean_lammean.txt"
        rf_lst = np.loadtxt(directory + rf_name, skiprows=2, usecols=1)  # uses default AirClim output format
        dt_lst = np.loadtxt(directory + dt_name, skiprows=2, usecols=1)  # uses default AirClim output format
        rf_arr[i + 1, :] = rf_lst  # i+1 because i=0 is the total
        dt_arr[i + 1, :] = dt_lst
    # Add totals
    rf_arr[0, :] = np.sum(rf_arr[:, :], axis=0)
    dt_arr[0, :] = np.sum(dt_arr[:, :], axis=0)
    return rf_arr, dt_arr, years


def rf_co2eq(H, forcing_profiles, temperature_profiles, years, CO2_emissions):
    """Calculate CO2eq emissions using Radiative Forcing (RF_H)

    Args:
        H (_float_): Time horizon [years]
        forcing_profiles (_np.ndarray_): RF values from AirClim
        temperature_profiles (_np.ndarray_): dT values from AirClim
        years (_np.ndarray_): Array of response years from AirClim
        CO2_emissions (_np.ndarray_): Array of yearly CO2 emissions

    Returns:
        rf_CO2_eq (_np.ndarray_): Array of CO2eq emissions 
        rf_years_idx (_np.ndarray_): Array of corresponding indicies in years
    """
    rf_lst = forcing_profiles
    rfr_lst = rf_lst[:, H:-1] / rf_lst[1, H:-1]
    rf_CO2_eq = rfr_lst * CO2_emissions[:len(years)-H-1]
    rf_years_idx = range(len(years) - H - 1)  # minus 1 because of an error in AirClim
    return rf_CO2_eq, rf_years_idx


def gwp_co2eq(H, forcing_profiles, temperature_profiles, years, CO2_emissions):
    """Calculate CO2eq emissions using the Global Warming Potential (GWP_H)

    Args:
        H (_float_): Time horizon [years]
        forcing_profiles (_np.ndarray_): RF values from AirClim
        temperature_profiles (_np.ndarray_): dT values from AirClim
        years (_np.ndarray_): Array of response years from AirClim
        CO2_emissions (_np.ndarray_): Array of yearly CO2 emissions

    Returns:
        gwp_CO2_eq (_np.ndarray_): Array of CO2eq emissions 
        gwp_years_idx (_np.ndarray_): Array of corresponding indicies in years
    """
    agwp_lst = np.array([[np.sum(forcing_profiles[j, i:i + H]) for i in range(len(years) - H)]
                         for j in range(7)])
    gwp_lst = agwp_lst / agwp_lst[1, :]
    gwp_CO2_eq = gwp_lst * CO2_emissions[range(len(years) - H)]
    gwp_years_idx = range(len(years) - H)
    return gwp_CO2_eq, gwp_years_idx


def egwp_co2eq(H, forcing_profiles, temperature_profiles, years, CO2_emissions):
    """Calculate CO2eq emissions using the Global Warming Potential including the efficacy (EGWP_H)

    Args:
        H (_float_): Time horizon [years]
        forcing_profiles (_np.ndarray_): RF values from AirClim
        temperature_profiles (_np.ndarray_): dT values from AirClim
        years (_np.ndarray_): Array of response years from AirClim
        CO2_emissions (_np.ndarray_): Array of yearly CO2 emissions

    Returns:
        gwp_CO2_eq (_np.ndarray_): Array of CO2eq emissions 
        gwp_years_idx (_np.ndarray_): Array of corresponding indicies in years
    """
    efficacies = np.array([1., 1.14, 1.37, 1.18, 0.59, 1.])  # efficacies from Ponater et al. (2006)
    aegwp_lst = np.array([[np.sum(forcing_profiles[j, i:i + H]) for i in range(len(years) - H)]
                         for j in range(7)])
    aegwp_lst[1:, :] = aegwp_lst[1:, :] * efficacies.reshape((6, 1))
    aegwp_lst[0, :] = np.sum(aegwp_lst[1:, :], axis=0)
    egwp_lst = aegwp_lst / aegwp_lst[1, :]
    egwp_CO2_eq = egwp_lst * CO2_emissions[range(len(years) - H)]
    egwp_years_idx = range(len(years) - H)
    return egwp_CO2_eq, egwp_years_idx


def gtp_co2eq(H, forcing_profiles, temperature_profiles, years, CO2_emissions):
    """Calculate CO2eq emissions using the Global Temperature-Change Potential (GTP_H)

    Args:
        H (_float_): Time horizon [years]
        forcing_profiles (_np.ndarray_): RF values from AirClim
        temperature_profiles (_np.ndarray_): dT values from AirClim
        years (_np.ndarray_): Array of response years from AirClim
        CO2_emissions (_np.ndarray_): Array of yearly CO2 emissions

    Returns:
        gtp_CO2_eq (_np.ndarray_): Array of CO2eq emissions 
        gtp_years_idx (_np.ndarray_): Array of corresponding indicies in years
    """
    agtp_lst = temperature_profiles
    gtp_lst = agtp_lst[:, H:] / agtp_lst[1, H:]
    gtp_CO2_eq = gtp_lst * CO2_emissions[:len(years)-H]
    gtp_years_idx = range(len(years) - H)
    return gtp_CO2_eq, gtp_years_idx
    

def atr_co2eq(H, forcing_profiles, temperature_profiles, years, CO2_emissions):
    """Calculate CO2eq emissions using the Average Temperature Response (ATR_H)

    Args:
        H (_float_): Time horizon [years]
        forcing_profiles (_np.ndarray_): RF values from AirClim
        temperature_profiles (_np.ndarray_): dT values from AirClim
        years (_np.ndarray_): Array of response years from AirClim
        CO2_emissions (_np.ndarray_): Array of yearly CO2 emissions

    Returns:
        atr_CO2_eq (_np.ndarray_): Array of CO2eq emissions 
        atr_years_idx (_np.ndarray_): Array of corresponding indicies in years
    """
    atr_lst = np.array([[np.sum(temperature_profiles[j, i:i + H]) / H
                         for i in range(len(years) - H)] for j in range(7)])
    atrp_lst = atr_lst / atr_lst[1, :]
    atr_CO2_eq = atrp_lst * CO2_emissions[range(len(years) - H)]
    atr_years_idx = range(len(years) - H)
    return atr_CO2_eq, atr_years_idx


def gwps_co2eq(H, forcing_profiles, temperature_profiles, years, CO2_emissions,
               agwph_co2_file="../data/MVMC/AGWPH_CO2_SSP2-45.txt"):
    """Calculate CO2eq emissions using the GWP*_H climate metric

    Args:
        H (_float_): Time horizon [years]
        forcing_profiles (_np.ndarray_): RF values from AirClim
        temperature_profiles (_np.ndarray_): dT values from AirClim
        years (_np.ndarray_): Array of response years from AirClim
        CO2_emissions (_np.ndarray_): Array of yearly CO2 emissions
        agwph_co2_file (_str_): Path to AGWP_CO2(H) file (AGWP as a function of H for CO2 for 1<=H<=100)

    Returns:
        gwps_co2eq (_np.ndarray_): Array of CO2eq emissions 
        yr_idx1 (_np.ndarray_): Array of corresponding indicies in years
    """
    dt = 20
    s = 0.25  # in line with Smith et al. (2021) for CH4
    agwph_co2 = np.loadtxt(agwph_co2_file)
    yr_idx1 = np.arange(dt, len(forcing_profiles[0, :]))
    agwp_co2 = agwph_co2[H - 1] * 1e9
    dFdt = np.subtract(forcing_profiles[:, np.arange(dt, len(years))],
                       forcing_profiles[:, np.arange(0, len(years) - dt)]) / dt / 1000.
    F_av = np.array([[sum(forcing_profiles[j, i - dt + 1: i + 1]) / dt
                      for i in np.arange(dt, len(years))]
                     for j in range(7)]) / 1000.
    g = (1 - np.exp(-s / (1 - s))) / s  # Extension by Smith et al. (2021)
    gwps_co2eq = g * ((1 - s) * dFdt * H / agwp_co2 + s * F_av / agwp_co2)
    gwps_co2eq[1, :] = CO2_emissions[yr_idx1]  # Add co2 emissions directly
    gwps_co2eq[0, :] = np.sum(gwps_co2eq[1:, :], axis=0)
    return gwps_co2eq, yr_idx1


def egwps_co2eq(H, forcing_profiles, temperature_profiles, years, CO2_emissions, 
                agwph_co2_file="../data/MVMC/AGWPH_CO2_SSP2-45.txt"):
    """Calculate CO2eq emissions using the EGWP*_H (Effective GWP*) climate metric

    Args:
        H (_float_): Time horizon [years]
        forcing_profiles (_np.ndarray_): RF values from AirClim
        temperature_profiles (_np.ndarray_): dT values from AirClim
        years (_np.ndarray_): Array of response years from AirClim
        CO2_emissions (_np.ndarray_): Array of yearly CO2 emissions
        agwph_co2_file (_str_): Path to AGWP_CO2(H) file (AGWP as a function of H for CO2 for 1<=H<=100)

    Returns:
        gwps_co2eq (_np.ndarray_): Array of CO2eq emissions 
        yr_idx1 (_np.ndarray_): Array of corresponding indicies in years
    """
    dt = 20
    s = 0.25  # in line with Smith et al. (2021) for CH4
    efficacies = np.array([1., 1.14, 1.37, 1.18, 0.59, 1.])  # Ponater et al. (2006)
    agwph_co2 = np.loadtxt(agwph_co2_file)
    yr_idx1 = np.arange(dt, len(forcing_profiles[0, :]))
    agwp_co2 = agwph_co2[H - 1] * 1e9
    dFdt = np.subtract(forcing_profiles[:, np.arange(dt, len(years))],
                       forcing_profiles[:, np.arange(0, len(years) - dt)]) / dt / 1000.
    F_av = np.array([[sum(forcing_profiles[j, i - dt + 1: i + 1]) / dt
                      for i in np.arange(dt, len(years))]
                     for j in range(7)]) / 1000.
    g = (1 - np.exp(-s / (1 - s))) / s  # Extension by Smith et al. (2021)
    gwps_co2eq = g * ((1 - s) * dFdt * H / agwp_co2 + s * F_av / agwp_co2)
    gwps_co2eq[1, :] = CO2_emissions[yr_idx1]  # Add co2 emissions directly
    gwps_co2eq[1:, :] = gwps_co2eq[1:, :] * efficacies.reshape((6, 1))
    gwps_co2eq[0, :] = np.sum(gwps_co2eq[1:, :], axis=0)
    return gwps_co2eq, yr_idx1

## Plot CO2eq emissions per climate metric

This section calculates CO2eq emissions for the CORSIA and FP2050 scenarios (Grewe et al., 2021) for each climate metric (Figure 2 in the linked paper). Uncomment the last line to save the figure to the `images` folder.

In [ ]:
# initialise figure
fig, axs = plt.subplots(1, 2, figsize=(7.5, 4), sharey=True)
met_co2eq_lst = [rf_co2eq, gwp_co2eq, egwp_co2eq, gtp_co2eq, atr_co2eq, gwps_co2eq, egwps_co2eq]
metrics = ["S-RFI", "S-GWP", "S-EGWP", "S-GTP", "S-rATR", "S-GWP*", "S-EGWP*"]
markers = ["D", "o", "s", "h", "^", "*", "X"]
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']
linestyles = ['-', '--', '--', '-.', ':', (0, (3, 1, 1, 1, 1, 1)), (0, (3, 1, 3, 1, 1, 1))]
markersizes = [3.5, 4, 4, 4, 4, 6, 5]
markeverys = [(35, 20), (40, 20), (40, 20), (50, 20), (50, 20), (0, 20), (10, 20)]  # different spacing for clarity

# load scenarios
scenarios = ["CORSIA", "FP2050"]  # indexes must match columns of E_bg_new_scen!
scenario_idxs = [2, 3]
SSP = "SSP2-45"
scenario_data = (np.loadtxt("../data/CO2eq/E_bg_new_scen.txt", skiprows=2))[:, scenario_idxs]
scenario_years = np.loadtxt("../data/CO2eq/E_bg_new_scen.txt", skiprows=2, usecols=0)

# create figure
for i, scenario in enumerate(scenarios):
    # plot CO2eq
    ax = axs[i]
    ax.axhline(0, color="k", linewidth=0.5, linestyle="dashed")
    for j, met in enumerate(metrics):
        folder = "../data/CO2eq/{}/{}/".format(scenario, SSP)
        rf_arr, dt_arr, years = load_rf_dt(folder)
        max_yr_idx = int((np.where(years == 2100))[0])
        co2_emis = (np.loadtxt(folder + "CO2_emis.txt"))[:, 1]
        met_res, met_idx = met_co2eq_lst[j](100, rf_arr, dt_arr, years, co2_emis)
        
        for k in [0]:  # just plot total
            ax.plot(years[met_idx], met_res[k, :],
                    color=colors[j], linestyle=linestyles[j], linewidth=1.2,
                    marker=markers[j], markevery=markeverys[j], markersize=markersizes[j] * 0.9,
                    label=met if k == 0 else "_")

    # set labels and limits
    ax.set_xlim([1940, 2100])
    ax.set_ylim([-6000, 8000])
    ax.set_ylabel("Yearly total CO$_2$-eq Emissions [Tg CO$_2$-eq]")
    ax.set_xlabel("Year")
    ax.text(0.05, 0.92, scenario, fontsize=12, transform=ax.transAxes)
    ax.grid(linewidth=0.5)
    if i == 0:
        ax.set_xticks([1940, 1960, 1980, 2000, 2020, 2040, 2060, 2080])
    else:
        ax.set_ylabel("")
        ax.set_xticks([1940, 1960, 1980, 2000, 2020, 2040, 2060, 2080, 2100])
    
    # temperature axis
    ax_t = ax.twinx()
    ax_t.plot(years[:max_yr_idx], dt_arr[0, :max_yr_idx],
              color="k", label="Temperature", linewidth=1)
    ax_t.set_ylim([0, 700])
    if i == 1:
        ax_t.set_yticks([0, 100, 200])
        ax_t.set_ylabel("Temperature [mK]")
        ax_t.yaxis.set_label_coords(1.12, 0.2, transform=ax_t.transAxes)
    else:
        ax_t.set_yticks([])
    
    # fuel axis
    ax_f = ax.twinx()
    ax_f.plot(scenario_years, scenario_data[:, i],
              color="r", linestyle="--",
              label="Fuel use", linewidth=1)
    ax_f.set_ylim([0, 7000])
    if i == 1:
        ax_f.set_yticks([1000, 2000])
        ax_f.yaxis.tick_left()
        ax_f.set_ylabel("Yearly fuel use [Tg]")
        ax_f.tick_params(axis="y", direction="in", pad=-28)
        ax_f.yaxis.set_label_coords(0.15, 0.22, transform=ax_f.transAxes)
    else:
        ax_f.set_yticks([])
    
    # create legend
    if i == 0:
        lines_co2eq, labels_co2eq = ax.get_legend_handles_labels()
        lines_t, labels_t = ax_t.get_legend_handles_labels()
        lines_f, labels_f = ax_f.get_legend_handles_labels()
        ax.legend(lines_co2eq, labels_co2eq, loc=(0.015, 0.21),
                  ncol=2, framealpha=1.0, fontsize=8.5, handlelength=3)
        ax_f.legend(lines_t + lines_f, labels_t + labels_f,
                    loc=(0.015, 0.09), framealpha=1.0, fontsize=8.5)

# create subfigure labels
axs[0].text(0.9, 0.92, "a", fontsize=16, transform=axs[0].transAxes)
axs[1].text(0.9, 0.92, "b", fontsize=16, transform=axs[1].transAxes)
axs[0].text(0.9, 0.04, "c", fontsize=16, transform=axs[0].transAxes)
axs[1].text(0.9, 0.04, "d", fontsize=16, transform=axs[1].transAxes)

# configure and save figure
fig.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0)
fig_savename = "../images/co2eq_tot_trajectories.png"
# fig.savefig(fig_savename, dpi=600)

## Comparison of GWP, EGWP and ATR CO2eq emissions

This section calculates CO2eq emissions for the CORSIA and FP2050 scenarios (Grewe et al., 2021) and all emission species for the GWP, EGWP and ATR (Extended Data Figure 2 in the linked paper). Uncomment the last line to save the figure to the `images` folder.

In [ ]:
# initialise figure
fig, axs = plt.subplots(1, 2, figsize=(10, 3.5), sharex=True, sharey=True)
met_co2eq_lst = [gwp_co2eq, egwp_co2eq, atr_co2eq]
metrics = ["GWP", "EGWP", "iGTP/rATR",]
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']
emission_labels = ["Total", r"CO$_2$", r"H$_2$O", r"O$_3$", r"CH$_4$", "Contrails", "PMO"]
markers = ["o", "s", "^"]
markeverys = [(0, 20), (0, 20), (10, 20)]  # different spacing for clarity
linestyles = ['-', '--', ':']

# set scenarios
scenarios = ["CORSIA", "FP2050"]
scenario_idxs = [2, 3]
SSP = "SSP2-45"
scenario_data = (np.loadtxt("../data/CO2eq/E_bg_new_scen.txt", skiprows=2))[:, scenario_idxs]
scenario_years = np.loadtxt("../data/CO2eq/E_bg_new_scen.txt", skiprows=2, usecols=0)

# plot
for i, scenario in enumerate(scenarios):
    ax = axs[i]
    for j, met in enumerate(metrics):
        folder = "../data/CO2eq/{}/{}/".format(scenario, SSP)
        rf_arr, dt_arr, years = load_rf_dt(folder)
        max_yr_idx = int((np.where(years == 2100))[0])
        co2_emis = (np.loadtxt(folder + "CO2_emis.txt"))[:, 1]
        met_res, met_idx = met_co2eq_lst[j](100, rf_arr, dt_arr, years, co2_emis)
        net_nox = met_res[3, :] + met_res[4, :] + met_res[6, :]
        for k in range(7):
            ax.plot(years[met_idx], met_res[k, :],
                    color=colors[k], linestyle=linestyles[j], linewidth=1,
                    marker=markers[j], markevery=markeverys[j], markersize=3)

# metric legend
met_legend_elements = [
    Line2D([0], [0], marker=markers[0], color="k", markerfacecolor="k", markersize=3,
           linestyle=linestyles[0], linewidth=1, label=metrics[0]),
    Line2D([0], [0], marker=markers[1], color="k", markerfacecolor="k", markersize=3,
           linestyle=linestyles[1], linewidth=1, label=metrics[1]),
    Line2D([0], [0], marker=markers[2], color="k", markerfacecolor="k", markersize=3,
           linestyle=linestyles[2], linewidth=1, label=metrics[2]),
]
met_legend = axs[0].legend(handles=met_legend_elements, ncol=2, loc="lower left", bbox_to_anchor=(0., 0.82))

# species legend
species_legend_elements = [
    Line2D([0], [0], color=color, linewidth=2, label=label) for color, label in zip(colors, emission_labels)
]
species_legend = axs[0].legend(handles=species_legend_elements, ncol=2, loc="lower left", bbox_to_anchor=(0., 0.49))
axs[0].add_artist(met_legend)

# labels and limits
axs[0].set_xlim([1940, 2100])
axs[0].set_ylim([-1500, 7000])
axs[0].set_ylabel('Yearly CO$_2$-eq Emissions [Tg CO$_2$-eq]')
axs[0].set_xlabel("Years")
axs[1].set_xlabel("Years")
axs[0].text(0.05, 0.05, "a - {}".format(scenarios[0]), transform=axs[0].transAxes, fontsize=12)
axs[1].text(0.05, 0.05, "b - {}".format(scenarios[1]), transform=axs[1].transAxes, fontsize=12)

# save figure
fig.tight_layout()
fig_savename = "../images/co2eq_e-gwpatr_trajectories.png"
# fig.savefig(fig_savename, dpi=600)